In [70]:
import numpy as np
import argparse
import cv2
import os
from datetime import datetime
from PIL import Image

In [71]:
def save_img(file_name, img, saveTo):
    if not os.path.exists(saveTo):
        os.makedirs(saveTo)
    img = img[:,:,::-1]
    img = Image.fromarray(img)

    try:
        img.save(saveTo + "/" + file_name + ".jpeg", resolution=100.0)
    except Exception as ex:
        print("Error while saving file.")
        print(ex)
        return False
    else: return True

In [80]:

def capture_images(camId = 0, path = f'{os.path.abspath(os.curdir)}\checkerboards_cal'):
    font                   = cv2.FONT_HERSHEY_DUPLEX
    position               = (15,50)
    fontScale              = 1.3
    fontColor              = (255,255,255)
    lineType               = 2

    cap = cv2.VideoCapture(camId)
    while True:
        ret, frame = cap.read()
        if ret is False: 
            return
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (8, 6), None)
        if corners is not None:
            for i in corners:
                x,y = i.ravel()
                cv2.circle(frame,(x,y),5,(0,0,255),-1)
        key =  cv2.waitKey(30) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('s'):
            img = frame.copy()
            cv2.putText(frame,'To save click S, to pass P', 
                position, 
                font, 
                fontScale,
                fontColor,
                lineType)
            cv2.imshow('frame', frame)
            doSave = cv2.waitKey(0)  & 0xFF
            if doSave == ord('s'):
                gene_name = f'Checkerboard_ + {datetime.now():%Y%m%d_%H%M%S}'
                res = save_img(gene_name, img, path) 
                if res is True: print(f"Saved: {os.path.abspath(os.curdir)}\\checkerboards_cal\\{gene_name}")
                else: continue
        cv2.imshow('frame', frame)
    cap.release
    cv2.destroyAllWindows()
        

In [ ]:
capture_images(1)

Saved: C:\Users\barte\Documents\Studia VII\Image_processing\Calibrate and sketch view\checkerboards_cal\Checkerboard_ + 20181118_163534
Saved: C:\Users\barte\Documents\Studia VII\Image_processing\Calibrate and sketch view\checkerboards_cal\Checkerboard_ + 20181118_163540
Saved: C:\Users\barte\Documents\Studia VII\Image_processing\Calibrate and sketch view\checkerboards_cal\Checkerboard_ + 20181118_163609
Saved: C:\Users\barte\Documents\Studia VII\Image_processing\Calibrate and sketch view\checkerboards_cal\Checkerboard_ + 20181118_163622
Saved: C:\Users\barte\Documents\Studia VII\Image_processing\Calibrate and sketch view\checkerboards_cal\Checkerboard_ + 20181118_163631
Saved: C:\Users\barte\Documents\Studia VII\Image_processing\Calibrate and sketch view\checkerboards_cal\Checkerboard_ + 20181118_163640
Saved: C:\Users\barte\Documents\Studia VII\Image_processing\Calibrate and sketch view\checkerboards_cal\Checkerboard_ + 20181118_163654
Saved: C:\Users\barte\Documents\Studia VII\Image

In [ ]:
os.path.abspath(os.curdir)